In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all ifiles under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cv-course-models/cv-course-models/simple-cnn.ipynb
/kaggle/input/cv-course-models/cv-course-models/minivgg.md
/kaggle/input/cv-course-models/cv-course-models/simple-cnn-bn.ipynb
/kaggle/input/cv-course-models/cv-course-models/minivgg-bn.ipynb
/kaggle/input/cv-course-models/cv-course-models/vgg16.ipynb
/kaggle/input/cv-course-models/cv-course-models/inceptionv1.md
/kaggle/input/cv-course-models/cv-course-models/vgg16.md
/kaggle/input/cv-course-models/cv-course-models/minivgg-bn.md
/kaggle/input/cv-course-models/cv-course-models/simple-cnn-bn.md
/kaggle/input/cv-course-models/cv-course-models/inceptionv1.ipynb
/kaggle/input/cv-course-models/cv-course-models/simple-cnn.md
/kaggle/input/cv-course-models/cv-course-models/minivgg.ipynb
/kaggle/input/cv-course-models/cv-course-models/simple-cnn-untrained/saved_model.pb
/kaggle/input/cv-course-models/cv-course-models/simple-cnn-untrained/variables/variables.index
/kaggle/input/cv-course-models/cv-course-models/simple-cnn-untraine

In [2]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential



In [3]:
train = pd.read_csv('../input/dataset/dataset/Train.csv')
test = pd.read_csv('../input/dataset/dataset/Test.csv')

In [ ]:
print(len(train))

In [ ]:
train.head()

In [4]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model

In [ ]:
train.head()

In [8]:


from PIL import Image
img = Image.open('../input/dataset/dataset/train images/Images/id_02amazy34fgh2.jpg')
nparr  = np.asarray(img)



In [ ]:
nparr.shape

In [ ]:


train.iloc[2]['Image_id']



In [5]:
from sklearn.model_selection import train_test_split

train,validate = train_test_split(train,test_size=0.2)

new_img_size = (128,128)



In [6]:
validate.Label.value_counts()

0    167
1    157
Name: Label, dtype: int64

In [9]:
images_train=[]
img = Image.open('../input/dataset/dataset/train images/Images/'+str(train.iloc[0]['Image_id']))
img=img.resize(new_img_size)
images_train = np.array([np.asarray(img)])

for i in range(1,len(train)):
    img = Image.open(f'../input/dataset/dataset/train images/Images/'+str(train.iloc[i]['Image_id']))
    img = img.resize(new_img_size)
    images_train = np.concatenate((images_train,np.array([np.asarray(img)])),axis=0)

In [ ]:
print(images_train.shape)

In [ ]:
print(images_train[0].shape)

In [ ]:
images_train[0]

In [10]:
img = Image.open('../input/dataset/dataset/train images/Images/'+str(validate.iloc[0]["Image_id"]))
img=img.resize(new_img_size)
images_validate = np.array([np.asarray(img)])
for i in range(1,validate.shape[0]):
    img = Image.open('../input/dataset/dataset/train images/Images/'+str(validate.iloc[i]["Image_id"]))
    img = img.resize(new_img_size)
    images_validate = np.concatenate((images_validate,np.array([np.asarray(img)])),axis=0)

In [11]:


img = Image.open(f'../input/dataset/dataset/train images/Images/'+str(test.iloc[0]["Image_id"]))
img=img.resize(new_img_size)
images_test = np.array([np.asarray(img)])
for i in range(1,test.shape[0]):
    img = Image.open('../input/dataset/dataset/train images/Images/'+str(test.iloc[i]["Image_id"]))
    img = img.resize(new_img_size)
    images_test = np.concatenate((images_test,np.array([np.asarray(img)])),axis=0)



In [12]:
print("Training set is of shape : ",images_train.shape)
print("Validation set is of shape : ",images_validate.shape)
print("Test set is of shape : ",images_test.shape)


Training set is of shape :  (1295, 128, 128, 3)
Validation set is of shape :  (324, 128, 128, 3)
Test set is of shape :  (1080, 128, 128, 3)


# **Model**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
# these are a new feature in TF 2.2
from tensorflow.keras.layers.experimental import preprocessing


pretrained_base = tf.keras.models.load_model(
    '../input/cv-course-models/cv-course-models/vgg16-pretrained-base',
)
pretrained_base.trainable = False

model = keras.Sequential([
    # Preprocessing
    preprocessing.RandomFlip('horizontal'), # flip left-to-right
    preprocessing.RandomContrast(0.5), # contrast change by up to 50%
    # Base
    pretrained_base,
    # Head
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

history = model.fit(
    images_train,
    train['Label'],validation_data=(images_validate,validate['Label']),
    epochs=50,
    verbose=1,
)

In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [18]:
model_19 = VGG19(weights = "imagenet", include_top=False,input_shape=(128,128,3))
model_19.trainable = False
flat19 = Flatten()(model_19.layers[-1].output)
class19 = Dense(128,activation="relu")(flat19)
#class2 = Dense(50,activation="relu")(class1)
#class3 = Dense(1024,activation="relu")(class2)
output19 = Dense(1,activation="sigmoid")(class19)
model_19 = Model(inputs=model_19.inputs, outputs=output19)

In [19]:
adam_opt = Adam()
model_loss = "binary_crossentropy"
early_stopping = EarlyStopping(patience=50, restore_best_weights=True)
model_check_point_19 = ModelCheckpoint(filepath= './vgg19', 
                                    monitor='val_loss', verbose=1, 
                                    save_best_only=True,
                                    save_weights_only=False, 
                                    mode='auto', save_freq='epoch')

In [20]:
model_19.compile(optimizer=adam_opt,loss=model_loss,metrics=["binary_accuracy"])

In [ ]:
import time

In [21]:
history_19 = model_19.fit(images_train,
    train['Label'],validation_data=(images_validate,validate['Label']),epochs=50,callbacks=[early_stopping,model_check_point_19],verbose=1)


Epoch 1/50
31/41 [=====================>........] - ETA: 31s - loss: 1.7443 - binary_accuracy: 0.8942

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16


model16 = VGG16()

In [ ]:
import time
from tensorflow.keras.optimizers import Adam

In [ ]:
model_16 = VGG16(weights = "imagenet", include_top=False,input_shape=(224,224,3))
model_16.trainable = False
flat1 = Flatten()(model_16.layers[-1].output)
class1 = Dense(128,activation="relu")(flat1)
#class2 = Dense(50,activation="relu")(class1)
#class3 = Dense(1024,activation="relu")(class2)
output = Dense(1,activation="sigmoid")(class1)
model_16 = Model(inputs=model_16.inputs, outputs=output)

In [ ]:
adam_opt = Adam()
model_loss = "binary_crossentropy"
early_stopping = EarlyStopping(patience=200, restore_best_weights=True)
model_check_point_16 = ModelCheckpoint(filepath= './vgg16', 
                                    monitor='val_loss', verbose=1, 
                                    save_best_only=True,
                                    save_weights_only=False, 
                                    mode='auto', save_freq='epoch')

In [ ]:
model_16.compile(optimizer=adam_opt,loss=model_loss,metrics=["binary_accuracy"])

In [ ]:
history_16 = model_16.fit(images_train,
    train['Label'],validation_data=(images_validate,validate['Label']),epochs=200,callbacks=[early_stopping,model_check_point_16],verbose=1)
end_time = time.time()
print("Training time for VGG16: ", end_time - start_time,"secs")

 

In [ ]:
""""model = Sequential([
    layers.Rescaling(1./255.,input_shape=(224,224,3)),
    layers.Conv2D(8,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),
    layers.Conv2D(16,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),
    layers.Conv2D(32,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(256,activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(1,activation='sigmoid'),   
])
model.build()
"""



In [ ]:

"""

model.compile(
    optimizer='adam',
    loss = tf.losses.BinaryCrossentropy(),
    metrics=['binary_accuracy']
)
"""



In [ ]:
"""

batch_size = 32
epochs = 15

model.fit(
    images_train,
    train['Label'],
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(images_validate,validate['Label'])
)
"""


In [ ]:


data_augmentaion = Sequential([
    layers.RandomRotation(factor=(-0.2,0.2),seed = 123),
    layers.RandomZoom(0.1),
])



In [ ]:
"""

model = Sequential([
    layers.Rescaling(1./255.,input_shape=(224,224,3)),
    data_augmentaion,
    layers.Conv2D(8,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),
    layers.Conv2D(16,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.2),
    layers.Conv2D(32,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Dropout(0.4),
    layers.Flatten(),
    layers.Dense(256,activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(1,activation='sigmoid'),   
])
model.build()

"""


In [ ]:
"""


model.compile(
    optimizer='adam',
    loss = tf.losses.BinaryCrossentropy(),
    metrics=['binary_accuracy']
)

batch_size = 32
epochs = 100

history = model.fit(
    images_train,
    train['Label'],
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(images_validate,validate['Label'])
)

"""


In [22]:
train_acc = history.history['binary_accuracy']
validate_acc = history.history['val_binary_accuracy']
epoch_ = range(1,epochs+1)

plt.plot(epoch_,train_acc,'r',label='Training')
plt.plot(epoch_,validate_acc,'b',label='Validation')
plt.legend()
plt.show()


NameError: name 'history' is not defined

# Prediction time!

In [24]:


predictions = model_19.predict(images_test)



In [25]:
predictions

array([[6.1303072e-28],
       [1.6747091e-24],
       [1.0000000e+00],
       ...,
       [3.6600172e-27],
       [1.0000000e+00],
       [9.9959195e-01]], dtype=float32)

In [26]:


predictions.shape



(1080, 1)

In [28]:
cm_predictions = np.argmax(predictions, axis = -1)

In [31]:
for i in range(len(predictions)):
    print(predictions[i])
    

[6.130307e-28]
[1.6747091e-24]
[1.]
[2.334089e-23]
[0.]
[1.2047419e-20]
[1.]
[0.00032392]
[5.4366224e-21]
[0.]
[1.]
[2.7713766e-06]
[0.00011142]
[1.893718e-30]
[2.858258e-16]
[1.]
[0.99999976]
[1.]
[2.708247e-19]
[0.]
[2.0267317e-23]
[8.0746116e-25]
[3.710449e-31]
[2.8791384e-38]
[1.]
[5.70818e-25]
[1.]
[9.3067513e-29]
[0.07918158]
[0.]
[7.6081287e-29]
[2.9025508e-30]
[3.131017e-14]
[4.2819162e-27]
[3.0295194e-34]
[1.]
[2.0232203e-12]
[1.]
[1.]
[1.]
[0.]
[1.]
[3.902681e-15]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.898974e-11]
[1.]
[0.9574011]
[1.]
[1.]
[2.4817311e-06]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.00061852]
[1.]
[2.7053193e-38]
[0.23877808]
[3.3475986e-17]
[1.]
[4.3849543e-24]
[1.692041e-05]
[1.]
[1.]
[1.2354144e-38]
[1.]
[1.]
[1.]
[1.]
[1.]
[6.3457135e-20]
[2.0213593e-29]
[1.]
[6.040638e-11]
[1.]
[0.]
[1.]
[5.767287e-36]
[1.3482473e-37]
[8.584048e-31]
[7.441932e-36]
[0.652099]
[0.9904063]
[1.]
[2.0733457e-16]
[4.781771e-24]
[0.]
[1.]
[0.99688816]
[1.1164601e-33]
[8.9115205e-20]
[2.977282e-

# **Submission**

In [ ]:


predictions = pd.DataFrame(np.round(predictions,2))
submission = pd.concat([test,predictions],axis=1)



In [ ]:
submission

In [ ]:
submission.rename(columns = {0:'Target', 'Image_id':'Image_ID'}, inplace = True)

In [ ]:
submission.tail(5)

In [ ]:
label_encode = {0:"healthy",1:"multiple_diseases",2:"rust",3:"scab"}

In [ ]:


submission.to_csv("submission.csv",index=False)

